#Import and reshape images

In [1]:
from keras.datasets import mnist
from keras import models
from keras import layers

(train_images_original, train_labels_original), (test_images_original, test_labels_original) = mnist.load_data()

import matplotlib.pyplot as plt
import numpy as np

train_images = train_images_original.reshape((60000, 28 * 28))
#train_images = train_images.astype('float32') / 255

test_images = test_images_original.reshape((10000, 28 * 28))
#test_images = test_images.astype('float32') / 255

next_label = 2

Using TensorFlow backend.


11493376/11490434 [==============================] - 1s 0us/step


#Number of regions

In [0]:
def flood_fill(img, row, col, label):
  if row < 0 or row > 27 or col < 0 or col > 27:
    return
  if img[row][col]==1:
    return
  if img[row][col] >1:
    return

  if img[row][col] == 0:
    img[row][col] = label
    flood_fill(img, row-1, col, label)
    flood_fill(img, row+1, col, label)
    flood_fill(img, row, col-1, label)
    flood_fill(img, row, col+1, label)
  
  global next_label
  next_label = label+1

def make_bw(img):
    a=np.where(img <100, 0, 1)
    return a

def number_regions(img):
  new_image = make_bw(img)
  global next_label
  next_label=2
  row = img.shape[0]
  cols = img.shape[1]

  for r in range(0, row-1):
    for c in range(0, cols-1):
      flood_fill(new_image, r, c, next_label)
  return (next_label - 2)




#Find width

In [0]:
def width(img):
  row = img.shape[0]
  cols = img.shape[1]
  first = 29
  last = 0

  for r in range(0, row-1):
    for c in range(0, cols-1):
      if(img[r][c]>0):
        if(first>c):
          first = c
        break

  for r in reversed(range(0, row-1)):
    for c in reversed(range(0, cols-1)):
      if(img[r][c]>0):
        if(last<c):
          last = c
        break
  
  return last - first + 1


#Find height

In [0]:
def height(img):
  row = img.shape[0]
  cols = img.shape[1]
  first = 29
  last = 0

  for c in range(0, row-1):
    for r in range(0, cols-1):
      if(img[r][c]>0):
        if(first>r):
          first = r
        break

  for c in reversed(range(0, row-1)):
    for r in reversed(range(0, cols-1)):
      if(img[r][c]>0):
        if(last<r):
          last = r
        break
  
  return last - first + 1

#Concatinate the values

In [0]:
HC_train = np.zeros((60000, 3), int)
HC_test = np.zeros((10000, 3), int)


#Add the handcrafted values to the train set
for i in range(0, 60000):
  a=number_regions(train_images_original[i])
  b=width(train_images_original[i])
  c=height(train_images_original[i])
  bc = np.array([a, b ,c])
  HC_train[i] = bc

#Add the handcrafted values to the test set
for i in range(0, 10000):
  a=number_regions(test_images_original[i])
  b=width(train_images_original[i])
  c=height(train_images_original[i])
  bc = np.array([a, b ,c])
  HC_test[i] = bc


#Stack the new data with the flattened data

train_image_hc = np.column_stack((train_images, HC_train))
test_image_hc = np.column_stack((test_images, HC_test))

#Debug and show image pixel data

#print('\n'.join([''.join(['{:4}'.format(item) for item in row]) 
      #for row in train_images_original[0]]))

#Single dense layer

In [10]:
import tensorflow as tf
from tensorflow import keras

trainable_image = train_image_hc / 255.0
testable_image = test_image_hc / 255.0

#setup

model = keras.Sequential([
    keras.layers.Dense(10, activation=tf.nn.softmax)
])

# compile the model

model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

# train the model
epochs = 10
history = model.fit(trainable_image, 
                      train_labels_original, 
                      epochs=epochs,  
                      validation_data=(testable_image, test_labels_original))

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 4s 69us/sample - loss: 0.4648 - acc: 0.8793 - val_loss: 0.3066 - val_acc: 0.9147
Epoch 2/10
60000/60000 [==============================] - 4s 67us/sample - loss: 0.3025 - acc: 0.9166 - val_loss: 0.2851 - val_acc: 0.9199
Epoch 3/10
60000/60000 [==============================] - 4s 67us/sample - loss: 0.2822 - acc: 0.9220 - val_loss: 0.2770 - val_acc: 0.9233
Epoch 4/10
60000/60000 [==============================] - 4s 67us/sample - loss: 0.2718 - acc: 0.9241 - val_loss: 0.2709 - val_acc: 0.9243
Epoch 5/10
60000/60000 [==============================] - 4s 67us/sample - loss: 0.2650 - acc: 0.9262 - val_loss: 0.2670 - val_acc: 0.9269
Epoch 6/10
60000/60000 [==============================] - 4s 67us/sample - loss: 0.2600 - acc: 0.9277 - val_loss: 0.2638 - val_acc: 0.9266
Epoch 7/10
60000/60000 [==============================] - 4s 67us/sample - loss: 0.2564 - acc: 0.9291 - val_loss: 0.